# E-ID tutorial
This Jupyter notebook explores the basic components of E-ID. It focuses, in particular, on the cryptographic scheme BBS+ and how it's used to ensure the privacy requirements of E-ID.
These requirements include:
1. Selective Disclosure
2. Unlinkability
3. Zero-Knowledge Proofs

To understand these concepts in more detail, please refer to our blog post on [E-ID infrastructure](https://c4dt.epfl.ch/article/the-swiss-confederation-e-id-public-sandbox-trust-infrastructure-part-2/)
## Requirements
1. Typescript
2. [tslab](https://github.com/yunabe/tslab). This is the Typescript kernel for Jupyter lab.
3. [dock.io Typescript crypto lib](https://github.com/docknetwork/crypto-wasm-ts)

In [3]:
import { initializeWasm } from '@docknetwork/crypto-wasm-ts'
await initializeWasm();

# Explanation of what follows

Identities in JSON, plus a schema...

In [4]:
const JOHN_SNOW_BANK_IDENTITY = {
  fname: 'John',
  lname: 'Snow',
  highly_secretive: {
    secret: 'super-duper-secret',
    email: 'john.snow@example.com',
    SSN: '123-456789-0',
    'user-id': 'user:USR-22201'
  },
  location: {
    country: 'CH',
    city: 'Lausanne',
    address: 'C4DT, station 14'
  },
  timeOfBirth: 1662010849619,
  physical: {
    height: 170,
    weight: 78,
    race: 'dark elf'
  },
  score: 200
};

const SCHEMA = {
  fname: null,
  lname: null,
  highly_secretive: {
    secret: null,
    email: null,
    SSN: null,
    'user-id': null
  },
  location: {
    country: null,
    city: null,
    address: null
  },
  timeOfBirth: null,
  physical: {
    height: null,
    weight: null,
    race: null
  },
  score: null
};

In [5]:
const stringToBytes = (str: string) => Uint8Array.from(Buffer.from(str, "utf-8"));

In [6]:
// Generating a keypair once signature parameters are created.
import { BBSPlusKeypairG2, BBSPlusSignatureParamsG1 } from '@docknetwork/crypto-wasm-ts';

// Deterministically generated params
const label = stringToBytes("Awesome key pair");
const cryptoParams = BBSPlusSignatureParamsG1.generate(14, label);

const keypair = BBSPlusKeypairG2.generate(cryptoParams);
const sk = keypair.secretKey;
const pk = keypair.publicKey;

In [7]:
import { Encoder, EncodeFunc } from '@docknetwork/crypto-wasm-ts'
import { BBSPlusSignatureG1 } from '@docknetwork/crypto-wasm-ts'

// The encoder has to be known and agreed upon by all system participants, i.e. signer, prover and verifier.
const encoders = new Map<string, EncodeFunc>();
encoders.set('score', Encoder.positiveIntegerEncoder());
const GlobalEncoder = new Encoder(encoders, Encoder.defaultEncodeFunc());

In [8]:
import { BBSPlusSignatureG1, getAdaptedSignatureParamsForMessages } from '@docknetwork/crypto-wasm-ts'


// The signing function will encode bytes to a field element as true is passed
const sig = BBSPlusSignatureG1.signMessageObject(JOHN_SNOW_BANK_IDENTITY, sk, label, GlobalEncoder);

// As the messages are not encoded, pass true to the verification function to make it encode messages before verifying the signature.
sig.signature.verifyMessageObject(JOHN_SNOW_BANK_IDENTITY, pk, label, GlobalEncoder);

const sigParams = getAdaptedSignatureParamsForMessages(cryptoParams, SCHEMA);

# Selective Disclosure 

In [9]:
// Both prover and verifier can independently create this struct
import { getRevealedAndUnrevealed } from '@docknetwork/crypto-wasm-ts'
const revealedNames = new Set(['fname', 'physical.height', 'location.city'])

// Prover prepares messages it wishes to reveal and hide.
const [revealedMsgs, unrevealedMsgs, revealedMsgsDecoded] = getRevealedAndUnrevealed(
    JOHN_SNOW_BANK_IDENTITY,
    revealedNames,
    GlobalEncoder
);

In [10]:
revealedMsgsDecoded

{
  fname: 'John',
  physical: { height: 170 },
  location: { city: 'Lausanne' }
}


In [11]:
import {Statement, Statements, MetaStatements, ProofSpec, Witness, Witnesses, CompositeProof} from '@docknetwork/crypto-wasm-ts';


// Create a BBS statement - the information we reveal
const statement1 = Statement.bbsPlusSignatureProverConstantTime(sigParams, revealedMsgs, false);
const statements = new Statements();
statements.add(statement1);

const proofSpec = new ProofSpec(statements, new MetaStatements(), [], stringToBytes("Awesome Proof - test 001"));

const witness1 = Witness.bbsPlusSignatureConstantTime(sig.signature, unrevealedMsgs, false);
const witnesses = new Witnesses();
witnesses.add(witness1);

const sharedNonce = stringToBytes('A unique nonce given by verifier');
const proof = CompositeProof.generate(proofSpec, witnesses, sharedNonce);

In [12]:
const verifierStatement1 = Statement.bbsPlusSignatureVerifierConstantTime(sigParams, pk, revealedMsgs, false);
const verifierStatements = new Statements();
verifierStatements.add(verifierStatement1);

const verifierProofSpec = new ProofSpec(verifierStatements, new MetaStatements(), [], stringToBytes("Awesome Proof - test 001"));

proof.verify(verifierProofSpec, sharedNonce)

{ verified: true, error: undefined }


# Zero-Knowledge proofs!
Here's we will provide a proof of the following: 
1. Selective disclosure of: User's firstName, lastName, location
2. Range proof: Score > 5

In [13]:
import { BoundCheckSnarkSetup, SetupParam } from '@docknetwork/crypto-wasm-ts';
const provingKey = BoundCheckSnarkSetup();
const snarkProvingKey = provingKey.decompress();
const snarkVerifyingKey = provingKey.getVerifyingKeyUncompressed();

const proverSetupParams: SetupParam[] = [];
proverSetupParams.push(SetupParam.legosnarkProvingKeyUncompressed(snarkProvingKey));

1


In [14]:
import { WitnessEqualityMetaStatement, getIndicesForMsgNames, MetaStatement, QuasiProofSpec, ProofSpec } from '@docknetwork/crypto-wasm-ts';


// Prover prepares messages it wishes to reveal and hide.
const revealedParametersB = new Set(['fname', 'lname', 'location.city'])

const [revealedMsgsB, unrevealedMsgsB, revealedMsgsDecodedB] = getRevealedAndUnrevealed(
    JOHN_SNOW_BANK_IDENTITY,
    revealedParametersB,
    GlobalEncoder
);

// Create a BBS signature
const statementB1 = Statement.bbsPlusSignatureProverConstantTime(sigParams, revealedMsgsB, false);
const statementB2 = Statement.boundCheckLegoProverFromSetupParamRefs(1, 1000, 0);
const statementsB = new Statements([statementB1, statementB2]);

const scoreFieldIndex = getIndicesForMsgNames(['score'], SCHEMA)[0]  // 12
const witnessEq = new WitnessEqualityMetaStatement();
witnessEq.addWitnessRef(0, scoreFieldIndex);  // addWitnessRef(statementId, dataPointIDInWitness)
witnessEq.addWitnessRef(1, 0);

const metaStatementsB = new MetaStatements();
metaStatementsB.addWitnessEquality(witnessEq);

const proofSpecB = new ProofSpec(statementsB, metaStatementsB, proverSetupParams);
proofSpecB.isValid();

true


In [15]:
const witnessB1 = Witness.bbsPlusSignatureConstantTime(sig.signature, unrevealedMsgsB, false);
const witnessB2 = Witness.boundCheckLegoGroth16(sig.encodedMessages['score']);
const witnessesB = new Witnesses([witnessB1, witnessB2]);

const proof = CompositeProof.generate(proofSpecB, witnessesB);
proof

CompositeProof {
  value: Uint8Array(1148) [
      2,   0,   0,   0,   0,   0,   0,   0,   0, 131, 208, 133,
     59, 130, 147,  28,  51,  66, 195,  71,  46, 190,  70,   5,
     94, 201, 172,  24,  13,   8,   4, 220, 167,  15,  59, 255,
     73, 233, 121,  81, 154, 114, 134, 201,  14, 217,  14, 176,
     23, 186,  30, 168,  34, 105, 128, 154,  74, 185,  51, 210,
    235, 163, 166, 171,  75,  30, 101,  76, 136,  10, 132, 142,
    116,   7, 166,  74,  94,  74,  48,  92, 152, 133, 218,  56,
     64, 216, 185, 126, 240, 108, 140, 143, 199,  54, 145, 247,
    139, 118,   3, 254,
    ... 1048 more items
  ]
}


In [16]:
import {encodeRevealedMsgs} from '@docknetwork/crypto-wasm-ts'; 


// Verifier 
const verifierSetupParams: SetupParam[] = [];
verifierSetupParams.push(SetupParam.legosnarkVerifyingKeyUncompressed(snarkVerifyingKey));

const revealedMsgs1FromVerifier = encodeRevealedMsgs(revealedMsgsDecodedB, SCHEMA, GlobalEncoder);
// checkMapsEqual(revealedMsgs, revealedMsgs1FromVerifier);

const statement1 = Statement.bbsPlusSignatureVerifierConstantTime(sigParams, pk, revealedMsgs1FromVerifier, false)
const statement2 = Statement.boundCheckLegoVerifierFromSetupParamRefs(1, 1000, 0);

const statementsVerifier = new Statements([statement1, statement2]);

const verifierWitnessEq = new WitnessEqualityMetaStatement();
verifierWitnessEq.addWitnessRef(0, getIndicesForMsgNames(['score'], SCHEMA)[0]);
verifierWitnessEq.addWitnessRef(1, 0);

const metaStatements = new MetaStatements();
metaStatements.addWitnessEquality(verifierWitnessEq);

const proofSpecVerifier = new ProofSpec(statementsVerifier, metaStatements, verifierSetupParams);
proofSpecVerifier.isValid();

proof.verify(proofSpecVerifier);

{ verified: true, error: undefined }


### Notes:
**Make sure to use TLS to send the information to the verifier. Data is not encrypted on its own**

In [134]:
 revealedMsgsDecodedB

{ fname: 'John', lname: 'Snow', location: { city: 'Lausanne' } }
